# 🛠️ JRA Scraper Fix Notebook

このノートブックは、JRAデータスクレイピングのエラーを解消するための修正版です。
まずはドライブをマウントし、必要なライブラリをセットアップします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import pandas as pd
import numpy as np
import time
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

# プロジェクトパスの設定 (環境に合わせて変更してください)
PROJECT_PATH = '/content/drive/MyDrive/dai-keiba'
sys.path.append(PROJECT_PATH)


## 1. RaceScraper Class definition

エラー回避のため、スクレイパークラスを直接定義します。

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
import re
import time
from datetime import datetime

class RaceScraper:
    def __init__(self):
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    def _get_soup(self, url):
        try:
            time.sleep(1) # Be polite
            response = requests.get(url, headers=self.headers, timeout=10)
            response.encoding = response.apparent_encoding
            if response.status_code == 200:
                return BeautifulSoup(response.text, 'html.parser')
        except Exception as e:
            print(f"Error fetching {url}: {e}")
        return None

    def get_past_races(self, horse_id, n_samples=5):
        """
        Fetches past n_samples race results for a given horse_id from netkeiba db.
        Returns a DataFrame of past races.
        """
        url = f"https://db.netkeiba.com/horse/result/{horse_id}/"
        soup = self._get_soup(url)
        if not soup:
            return pd.DataFrame()

        # The results are usually in a table with class "db_h_race_results"
        table = soup.select_one("table.db_h_race_results")
        if not table:
            # Try to find any table with "着順"
            tables = soup.find_all("table")
            for t in tables:
                if "着順" in t.text:
                    table = t
                    break
        
        if not table:
            return pd.DataFrame()

        # Parse table
        # We need to manually parse to get clean data and handle links if needed (though for past data, text is mostly fine)
        # pd.read_html is easier for the table
        try:
            df = pd.read_html(io.StringIO(str(table)))[0]
            
            # Basic cleaning
            df = df.dropna(how='all')
            
            # The columns in db.netkeiba are roughly:
            # 日付, 開催, 天気, R, レース名, 映像, 頭数, 枠番, ... 着順, ... 通過, ...
            
            # We want to keep: Date, Race Name, Course info, Rank, Time, Passing (Style)
            
            # Normalize column names (remove spaces/newlines)
            df.columns = df.columns.astype(str).str.replace(r'\s+', '', regex=True)

            # Filter rows that look like actual races (Date column exists)
            if '日付' in df.columns:
                df['date_obj'] = pd.to_datetime(df['日付'], format='%Y/%m/%d', errors='coerce')
                df = df.dropna(subset=['date_obj'])
                df = df.sort_values('date_obj', ascending=False)
                
            # Take top N
            if n_samples:
                df = df.head(n_samples)
            
            # Process Run Style (Leg Type)
            if '通過' in df.columns:
                df['run_style_val'] = df['通過'].apply(self.extract_run_style)
            else:
                df['run_style_val'] = 3 # Unknown

            # Extract/Rename Columns
            # We want: 日付, 開催, 天気, R, レース名, 映像, 頭数, 枠番, ... 着順, ... 通過, ...
            # Important: '上り' (3F), '馬体重', '騎手'
            
            # Map standard columns if they exist
            column_map = {
                '日付': 'date',
                '開催': 'venue',
                '天気': 'weather',
                'レース名': 'race_name',
                '着順': 'rank',
                '枠番': 'waku',
                '馬番': 'umaban',
                '騎手': 'jockey',
                '斤量': 'weight_carried',
                '馬場': 'condition', # 良/重/稍重 etc.
                'タイム': 'time',
                '着差': 'margin',
                '上り': 'last_3f',
                '通過': 'passing',
                '馬体重': 'horse_weight',
                'run_style_val': 'run_style',
                '単勝': 'odds',
                'オッズ': 'odds',
                '距離': 'raw_distance' # e.g. "芝1600"
            }
            
            # Rename available columns
            df.rename(columns=column_map, inplace=True)
            
            # Extract Surface and Distance from 'raw_distance'
            if 'raw_distance' in df.columns:
                def parse_dist(x):
                    if not isinstance(x, str): return None, None
                    # "芝1600", "ダ1200", "障3000"
                    # Sometimes "芝1600" or just "1600"
                    surf = None
                    dist = None
                    if '芝' in x: surf = '芝'
                    elif 'ダ' in x: surf = 'ダ'
                    elif '障' in x: surf = '障'
                    
                    # Extract number
                    match = re.search(r'(\d+)', x)
                    if match:
                        dist = int(match.group(1))
                    return surf, dist

                parsed = df['raw_distance'].apply(parse_dist)
                df['course_type'] = parsed.apply(lambda x: x[0])
                df['distance'] = parsed.apply(lambda x: x[1])
            else:
                df['course_type'] = None
                df['distance'] = None

            # Coerce numeric
            if 'rank' in df.columns:
                df['rank'] = pd.to_numeric(df['rank'], errors='coerce')
            
            if 'odds' in df.columns:
                 df['odds'] = pd.to_numeric(df['odds'], errors='coerce')
            
            # Fill missing
            for target_col in list(column_map.values()) + ['course_type', 'distance']:
                if target_col not in df.columns:
                    df[target_col] = None
                
            return df
            
        except Exception as e:
            print(f"Error parsing past races for {horse_id}: {e}")
            return pd.DataFrame()

    def extract_run_style(self, passing_str):
        """
        Converts passing order string (e.g., "1-1-1", "10-10-12") to run style (1,2,3,4).
        1: Nige (Escape) - Lead at 1st corner
        2: Senkou (Leader) - Within first ~4 or so
        3: Sashi (Mid) - Midpack
        4: Oikomi (Chaser) - Back
        Returns integer code.
        """
        if not isinstance(passing_str, str):
            return 3 # Default to Mid
            
        # Clean string "1-1-1" -> [1, 1, 1]
        try:
            cleaned = re.sub(r'[^0-9-]', '', passing_str)
            parts = [int(p) for p in cleaned.split('-') if p]
            
            if not parts:
                return 3
                
            first_corner = parts[0]
            
            # Heuristics
            if first_corner == 1:
                return 1 # Nige
            elif first_corner <= 4:
                return 2 # Senkou
            elif first_corner <= 9: # Assuming standard field size of 10-16, 9 is mid-ish limit? 
                # Actually "Sashi" is usually mid-rear. 
                # Let's say: 1=Lead, 2-4=Front, 5-10=Mid, >10=Back
                return 3 # Sashi
            else:
                return 4 # Oikomi
                
        except:
            return 3

    def scrape_race_with_history(self, race_id):
        """
        Detailed scraper that enters a race_result page, finding horse IDs, 
        then fetches history for each horse.
        Returns a dictionary or structured object with the race result + history.
        """
        url = f"https://race.netkeiba.com/race/result.html?race_id={race_id}"
        soup = self._get_soup(url)
        if not soup:
            return None
            
        # 1. Parse Main Result Table to get Horse IDs and basic result
        # Note: auto_scraper already does some of this, but we need Horse IDs specifically.
        # "All_Result_Table"
        
        result_data = []
        
        table = soup.find("table", id="All_Result_Table")
        if not table:
            return None
            
        rows = table.find_all("tr", class_="HorseList")
        
        print(f"Found {len(rows)} horses in race {race_id}. Fetching histories...")
        
        for row in rows:
            # Extract basic info
            rank_elem = row.select_one(".Rank")
            rank = rank_elem.text.strip() if rank_elem else ""
            
            horse_name_elem = row.select_one(".Horse_Name a")
            horse_name = horse_name_elem.text.strip() if horse_name_elem else ""
            horse_url = horse_name_elem.get("href") if horse_name_elem else ""
            
            # Extract ID from URL
            # https://db.netkeiba.com/horse/2018105027
            horse_id = None
            if horse_url:
                match = re.search(r'/horse/(\d+)', horse_url)
                if match:
                    horse_id = match.group(1)
            
            if not horse_id:
                print(f"  Skipping {horse_name} (No ID)")
                continue

            print(f"  Fetching history for {horse_name} ({horse_id})...")
            
            # 2. Get Past History
            df_past = self.get_past_races(horse_id, n_samples=5)
            
            # 3. Structure Data
            # converting df_past to a list of dicts or flattened fields
            history = []
            if not df_past.empty:
                for idx, r in df_past.iterrows():
                    # Extract relevant columns
                    # We need at least: Rank, RunStyle, Time(Seconds?), Pace?
                    # For now just dump raw-ish data
                    hist_item = {
                        "date": r.get('日付'),
                        "race_name": r.get('レース名'),
                        "rank": r.get('着順'),
                        "passing": r.get('通過'),
                        "run_style": r.get('run_style_val'),
                        "time": r.get('タイム'),
                        # Add more as needed for Feature Engineering
                    }
                    history.append(hist_item)
            
            entry = {
                "race_id": race_id,
                "horse_id": horse_id,
                "horse_name": horse_name,
                "rank": rank,
                "history": history
            }
            result_data.append(entry)
            
        return result_data

    def get_horse_profile(self, horse_id):
        """
        Fetches horse profile to get pedigree (Father, Mother, Grandfather(BMS)).
        Returns a dictionary or None.
        """
        # Use pedigree page for reliable bloodline data
        url = f"https://db.netkeiba.com/horse/ped/{horse_id}/"
        soup = self._get_soup(url)
        if not soup:
            return None
        
        # Parse Blood Table
        # table class="blood_table"
        
        data = {
            "father": "",
            "mother": "",
            "bms": ""
        }
        
        try:
            table = soup.select_one("table.blood_table")
            if table:
                rows = table.find_all("tr")
                # 5-generation table has 32 rows usually
                # Father at Row 0 (rowspan 16)
                # Mother at Row 16 (rowspan 16)
                
                if len(rows) >= 17:
                    # Father: Row 0, Col 0
                    r0 = rows[0].find_all("td")
                    if r0:
                        txt = r0[0].text.strip()
                        # Clean: "スクリーンヒーロー\n2004 栗毛..." -> "スクリーンヒーロー"
                        # Take first line
                        data["father"] = txt.split('\n')[0].strip()
                        
                    # Mother & BMS: Row 16
                    r16 = rows[16].find_all("td")
                    if len(r16) >= 2:
                        # Mother
                        m_txt = r16[0].text.strip()
                        data["mother"] = m_txt.split('\n')[0].strip()
                        
                        # BMS (Mother's Father)
                        bms_txt = r16[1].text.strip()
                        data["bms"] = bms_txt.split('\n')[0].strip()
                        
        except Exception as e:
            print(f"Error parsing profile for {horse_id}: {e}")
            
        return data

    def get_race_metadata(self, race_id):
        """
        Fetches metadata for a specific race ID from Netkeiba.
        Returns dict with: race_name, date, venue, course_type, distance, weather, condition, turn
        """
        url = f"https://race.netkeiba.com/race/result.html?race_id={race_id}"
        soup = self._get_soup(url)
        if not soup:
            return None
            
        data = {
            "race_name": "",
            "date": "",
            "venue": "",
            "course_type": "",
            "distance": "",
            "weather": "",
            "condition": "",
            "turn": "", # New: Dictionary key for turn direction
            "race_id": race_id
        }
        
        try:
            # Race Name
            title_elem = soup.select_one(".RaceName")
            if title_elem:
                data["race_name"] = title_elem.text.strip()
                
            # Date & Venue & Conditions
            # <div class="RaceData01">... 2023年1月5日 ... 1回中山1日 ...</div>
            # Content: "15:35発走 / 芝1600m (右 外) / 天候:晴 / 馬場:良"
            
            rd1 = soup.select_one(".RaceData01")
            
            if rd1:
                txt = rd1.text.strip()
                
                # Weather
                if "天候:晴" in txt: data["weather"] = "晴"
                elif "天候:曇" in txt: data["weather"] = "曇"
                elif "天候:小雨" in txt: data["weather"] = "小雨"
                elif "天候:雨" in txt: data["weather"] = "雨"
                elif "天候:雪" in txt: data["weather"] = "雪"
                
                # Condition
                if "馬場:良" in txt: data["condition"] = "良"
                elif "馬場:稍" in txt: data["condition"] = "稍重" # Covers 稍重
                elif "馬場:重" in txt: data["condition"] = "重"
                elif "馬場:不良" in txt: data["condition"] = "不良"
                
                # Course & Distance ("芝1600m")
                # Regex for "芝", "ダ", "障" followed by digits
                match = re.search(r'(芝|ダ|障)(\d+)m', txt)
                if match:
                    ctype_raw = match.group(1)
                    if ctype_raw == "芝": data["course_type"] = "芝"
                    elif ctype_raw == "ダ": data["course_type"] = "ダート"
                    elif ctype_raw == "障": data["course_type"] = "障害"
                    
                    data["distance"] = match.group(2)
                
                # Turn Direction ("右", "左", "直線")
                # Usually in parentheses like "(右)" or "(左)" or "(芝 左)"
                if "右" in txt: data["turn"] = "右"
                elif "左" in txt: data["turn"] = "左"
                elif "直線" in txt: data["turn"] = "直"

            # Date
            # Try finding date in Title or dedicated element
            date_elem = soup.select_one("dl#RaceList_DateList dd.Active") 
            if date_elem:
                 # Usually "1月5日(金)" - needs Year
                 # We can rely on the fact that race_id contains year (2025...)
                 # But let's look for YYYY年 in the whole text or title
                 pass
            
            # Fallback Date from Title Tag or Meta
            if not data["date"]:
                 meta_title = soup.title.text if soup.title else ""
                 match_date = re.search(r'(\d{4}年\d{1,2}月\d{1,2}日)', meta_title)
                 if match_date:
                     data["date"] = match_date.group(1)

        except Exception as e:
            print(f"Error parsing metadata for {race_id}: {e}")
            
        return data

if __name__ == "__main__":
    # Test
    scraper = RaceScraper()
    print("Running test...")
    # Example: Do Deuce (2019105219)
    # url = "https://db.netkeiba.com/horse/2019105219/"
    # print(f"Fetching {url}")
    df = scraper.get_past_races("2019105219")
    if df.empty:
        print("DF is empty. Checking raw soup for 'db_h_race_results'...")
        soup = scraper._get_soup(f"https://db.netkeiba.com/horse/result/2019105219/")
        if soup:
             t = soup.select_one("table.db_h_race_results")
             print(f"Selector 'table.db_h_race_results' found: {t is not None}")
             if not t:
                 print("Trying fallback 'table' with '着順'...")
                 tables = soup.find_all("table")
                 found = False
                 for i, tbl in enumerate(tables):
                     print(f"Table {i} classes: {tbl.get('class')}")
                     if "着順" in tbl.text or "着 順" in tbl.text or "日付" in tbl.text:
                         print("Found a table with '着順/日付'.")
                         # print(str(tbl)[:200])
                         t = tbl
                         found = True
                         break
                 if not found:
                     print("No table with '着順' found in soup.")
                     print("Soup snippet:", soup.text[:500])
                 else:
                    # Retry parsing with found table
                     try:
                        df = pd.read_html(str(t))[0]
                        print("Retry DF Head:")
                        print(df.head())
                     except Exception as e:
                        print(f"Retry parsing failed: {e}")
        else:
            print("Soup is None.")
    else:
        print(df.head())
        print("Columns:", df.columns)


## 2. Wrapper Functions

欠落していた関数を定義します。

In [ ]:
def fetch_race_horse_ids(race_id):
    """
    RaceScraperを使ってレース結果ページから馬IDを取得するラッパー関数
    Returns: (race_id, {horse_name: horse_id})
    """
    scraper = RaceScraper()
    # スクレイピング実行
    data = scraper.scrape_race_with_history(race_id)
    if not data:
        return None
    
    # {馬名: ID} の辞書を作成
    id_map = {}
    for entry in data:
        if entry.get('horse_name') and entry.get('horse_id'):
            id_map[entry['horse_name']] = entry['horse_id']
            
    return race_id, id_map

def fetch_horse_history(horse_id):
    """
    RaceScraperを使って馬の過去成績を取得するラッパー関数
    Returns: (horse_id, history_dataframe)
    """
    scraper = RaceScraper()
    df = scraper.get_past_races(horse_id, n_samples=5) # 直近5走のみ取得
    return horse_id, df


## 3. JRA Data Fix Script

以下を実行して欠損データを補完してください。

In [ ]:
# JRAコード
# 4.2 欠損データの補完 (JRA Version)

try:
    from tqdm.auto import tqdm
except ImportError:
    !pip install tqdm
    from tqdm.auto import tqdm

def fill_missing_past_data_jra_debug():
    csv_path = os.path.join(PROJECT_PATH, 'data', 'raw', 'database.csv')
    if not os.path.exists(csv_path):
        print(f'Error: {csv_path} not found.')
        return

    print(f'Reading {csv_path}...')
    df = pd.read_csv(csv_path, low_memory=False, dtype={'race_id': str, 'horse_id': str})
    
    # ID Cleaning
    if 'horse_id' in df.columns:
        df['horse_id'] = df['horse_id'].astype(str).str.replace(r'\.0$', '', regex=True)

    if '日付' in df.columns:
        df['date_dt'] = pd.to_datetime(df['日付'], format='%Y年%m月%d日', errors='coerce')
    else:
        print('Error: 日付 column not found.')
        return

    def save_df_safe(dataframe, msg=""):
        try:
             out_df = dataframe.drop(columns=['date_dt', 'date_obj'], errors='ignore')
             out_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
             print(f"  [Saved] {msg}")
        except Exception as e:
             print(f"  [Save Failed] {e}")

    # === Fill Past History Only ===
    unique_horses = df['horse_id'].dropna().unique()
    
    # Resume Logic
    if 'past_1_date' in df.columns:
        done_horses = df.loc[df['past_1_date'].notna(), 'horse_id'].unique()
        unique_horses = [h for h in unique_horses if h not in done_horses]
        
    print(f'Debugging history for {len(unique_horses)} horses...')

    horse_batch_size = 50 
    fields_map = {
        'date': 'date', 'rank': 'rank', 'time': 'time', 'race_name': 'race_name', 
        'last_3f': 'last_3f', 'horse_weight': 'horse_weight', 'jockey': 'jockey', 
        'condition': 'condition', 'odds': 'odds', 'weather': 'weather', 
        'distance': 'distance', 'course_type': 'course_type'
    }
    
    for k in fields_map.keys():
        for i in range(1, 6):
            col = f'past_{i}_{k}'
            if col not in df.columns: df[col] = None

    for i in range(0, len(unique_horses), horse_batch_size):
        batch_horses = unique_horses[i:i+horse_batch_size]
        print(f"Processing Batch {i//horse_batch_size + 1} ({len(batch_horses)} horses)...")
        
        history_store = {}
        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = {executor.submit(fetch_horse_history, hid): hid for hid in batch_horses}
            for future in tqdm(as_completed(futures), total=len(batch_horses), leave=False):
                try:
                    hid, hist_df = future.result()
                    if not hist_df.empty:
                        if 'date' in hist_df.columns:
                            hist_df['date_obj'] = pd.to_datetime(hist_df['date'], errors='coerce')
                            history_store[str(hid)] = hist_df
                except: pass
        
        modified_batch = False
        updates_count = 0
        
        if history_store:
            mask_batch = df['horse_id'].isin(history_store.keys())
            target_indices = df[mask_batch].index
            
            for idx in target_indices:
                hid = str(df.at[idx, 'horse_id'])
                current_date = df.at[idx, 'date_dt']
                
                if hid in history_store:
                    hist_df = history_store[hid]
                    if 'date_obj' not in hist_df.columns: continue
                    
                    past_races = hist_df[hist_df['date_obj'] < current_date].sort_values('date_obj', ascending=False).head(5)
                    
                    if not past_races.empty:
                        for j, (p_idx, p_row) in enumerate(past_races.iterrows()):
                            n = j + 1
                            for k, v in fields_map.items():
                                df.at[idx, f'past_{n}_{k}'] = p_row.get(v)
                                modified_batch = True
                        updates_count += 1
            
            if modified_batch:
                save_df_safe(df, f"Batch {i//horse_batch_size + 1}")
                import gc
                gc.collect()
            else:
                print("  ⚠️ No data updated in this batch.")
        else:
            print("  ⚠️ No history data fetched for this batch.")

    if 'date_dt' in df.columns: df.drop(columns=['date_dt'], inplace=True, errors='ignore')
    print('Done filling past data for JRA.')

fill_missing_past_data_jra_debug()
